In [ ]:
!apt-get update && apt-get install ffmpeg -y
!apt install tmux vim -y
!pip3 install moviepy openai python-dotenv pydub pytubefix openai-whisper llama-index llama-index-llms-openai llama-index-llms-ollama llama-index-embeddings-ollama 
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/install.sh | sh

# !chmod +x /usr/bin/ollama
# !useradd -r -s /bin/false -m -d /usr/share/ollama ollama
# !pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv dspy-ai graspologic


In [1]:
import os, pickle, gc, subprocess
from dotenv import load_dotenv
from video_transcription import split_audio_into_chunks, video_to_audio, transcribe_audio_chunks, get_transcription_model, download_video
from llama_index.core import Document
# import torch

load_dotenv("/workspace/repos/agentic-ai/.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

video_path="/workspace/data/video1512218125.mp4"
# video_path="https://www.youtube.com/watch?v=Z07Ewop7rQA"
audio_output_path="/workspace/data/video_audio.mp3"
transcribe_output_dir="/workspace/data"
file_save_path="/workspace/data/transcription.pkl"

In [5]:
if not os.path.exists(file_save_path):
    transcribe_model = get_transcription_model(open_source_model=True)

    print("Processing video...")
    if 'youtube' in video_path:
        print("   Downloading video from youtube...")
        download_video(video_url=video_path, audio_output_path=audio_output_path)
    else:
        video_to_audio(video_path=video_path, audio_output_path=audio_output_path)

    print("Splitting audio into chunks...")
    split_audio_into_chunks(audio_output_path=audio_output_path, transcribe_output_dir=transcribe_output_dir, max_chunk_size_mb=24)

    print("Transcribing audio chunks...")
    transcription = transcribe_audio_chunks(model=transcribe_model, chunk_dir="/workspace/data", file_save_path=file_save_path)

    del transcribe_model
    # torch.cuda.empty_cache()
    gc.collect()
else:
    with open(file_save_path, 'rb') as f:
        transcription = pickle.load(f)
    

    if isinstance(transcription[0], dict):
        documents = [Document(text=chunk["text"]) for chunk in transcription]

In [3]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI

# model_name, ctx_len = "gpt-4o", 128000
model_name, ctx_len = "llama3.1:latest", 128000

if model_name == "gpt-4o":
    llm = LOpenAI(model=model_name, max_tokens=4000)
else:
    try: sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: print(f"Error pulling model: Is the Ollama server running?\n{e}")
    addtion_kwargs = {"max_new_tokens": 2000}
    system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                request_timeout=1000.0, system_prompt=system_prompt, additional_kwargs=addtion_kwargs)
    print(llm.metadata)

Settings.llm = llm

LLMMetadata(context_window=128000, num_output=256, is_chat_model=True, is_function_calling_model=False, model_name='llama3.1:latest', system_role=<MessageRole.SYSTEM: 'system'>)

## Tree summarizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(llm=llm, verbose=True)
prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."

response = await summarizer.aget_response(prompt_summary, [doc.text for doc in transcription])

## LLM direct summarization

In [19]:
full_doc = " ".join([doc.text for doc in documents])
prompt_summary = f"You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it is factual, optimistic, and containing little information about change. The transcription is as follows:\n\n{full_doc}"

response = llm.complete(prompt_summary, max_tokens=5000)

In [21]:
print(response.text)

**MEMORANDUM**

**TO:** Investors and Stakeholders

**FROM:** AlphaTrAI Executive Team

**SUBJECT:** Corporate Update and Strategic Initiatives

We are pleased to provide a summary of our corporate update and strategic initiatives, as presented during a recent video conference.

**Corporate Update:**

In the face of an increasingly competitive tech startup landscape, AlphaTrAI has navigated significant challenges over the past 24-36 months. Despite this, we have not only survived but thrived, positioning ourselves for future growth and success. Our major pivot in 2022 involved transitioning from proprietary asset management to tech-enabled services, leveraging our expertise in wealth and asset management across various business areas.

**Strategic Initiatives:**

1. **Anchor Advisory Services Acquisition:** We are excited to announce the acquisition of Anchor Advisory Services, a leading provider of professional sales and distribution solutions. This strategic move will enhance our cap

In [ ]:
!cp '/workspace/repos/agentic-ai/PPM - MCG MADISON RIDGE DST.pdf' /workspace/data
!cp '/workspace/repos/agentic-ai/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx' /workspace/data
!apt update
!apt install tmux vim -y
!pip3 install llama-index llama-parse llama-index-embeddings-huggingface llama-index-llms-ollama
!pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv graspologic
!pip3 install flash-attn --no-build-isolation
!curl -fsSL https://ollama.com/download/ollama-linux-amd64.tgz | tar zx -C /usr


!ollama pull llama3.1:latest

In [ ]:

import json
import os
from dotenv import load_dotenv
from llama_index.llms.ollama import Ollama

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    Settings,
)

import nest_asyncio
nest_asyncio.apply()

load_dotenv('/workspace/repos/agentic-ai/.env')
access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

# model_name, num_ctx = "mistral-nemo", 128000
model_name, num_ctx = "llama3.1", 128000

addtion_kwargs = {"max_new_tokens": 3000}
system_prompt = "You are an expert in creating marketing materials for financial firms. You take corporate documents and turn them into marketing materials."
llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=num_ctx, model_type="chat", is_function_calling_model=False, 
             request_timeout=1000.0, system_prompt=system_prompt, additional_kwargs=addtion_kwargs)
Settings.llm = llm
llm.metadata

In [ ]:

Settings.chunk_size = 200
Settings.chunk_overlap = 50
embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
# embed_model_name = "BAAI/bge-small-en-v1.5"
# embed_model_name = "hkunlp/instructor-base"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name, device="cuda")

Settings.embed_model = embed_model


In [ ]:
from llamaindex_data_utils import extract_text_from_pdf
pdf_urls = ["/workspace/data/AIP - Capital Raise Proposal.pdf", "/workspace/data/Access Pre-IPOs Presentation June 2024.pdf"] 
llamaparse_kwargs={'result_type':"markdown", 'split_by_page':False}
documents = extract_text_from_pdf(pdf_urls, llama_api_key, llamaparse_kwargs=llamaparse_kwargs, save_json_path="/workspace/data/pdf_text.json")

In [ ]:
from rag_utils import create_llama_query_engine_rag
persist_dir="/workspace/data/vector_index"
query_engine = create_llama_query_engine_rag(llm, embed_model, persist_dir=persist_dir, documents=documents, vector_store_kwargs={})

In [ ]:
prompt = "You have been given a documents that describe the firm's investment strategy. Highlight portions of the document that include a unique opportunity to invest in pre-IPO companies and put investors on a level playing field with VCs and big institutional investors. You need to use this information to create a marketing brochure that will attract investors to the firm's products. The brochure should be informative, engaging, and persuasive. You must disclose in fine print that there are no guarantees, and investing is risky so nothing promissory."
response = query_engine.query(prompt)
print(response.response)

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
# splitter = SentenceSplitter(chunk_size=4000, chunk_overlap=100)
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    llm=llm,
    # transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

In [ ]:
prompt_summary = "Use specific details from the given documents to create a professional marketing letter that highlights portions of the context including a unique opportunity to invest in pre-IPO companies and put investors on a level playing field with VCs and big institutional investors, qualifications of the investment team, and why they should invest now. Use details from the documents, but do not make up information. Be professional and persuasive. At the end of the letter, you must disclose in fine print that there are no guarantees, and investing is risky."
response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents[1:]])

In [ ]:
print(response)

In [ ]:
for node in response.source_nodes:
    print(node.text)